<a href="https://colab.research.google.com/github/swwx0/financial_evaluation/blob/main/TrailingPE_ForwardPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import math
import glob
import os
from sklearn.preprocessing import MinMaxScaler
import unidecode
from sklearn.preprocessing import minmax_scale
import requests
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import csv

In [ ]:
data0= pd.read_excel(r"H:\Documents\ENSAE\1er semestre 2a\Python DS\projet\donnees_python.xlsx")
print(data0)

In [ ]:
data = data0.copy()

In [ ]:
data.columns

In [ ]:
data.iloc[31,:]

In [ ]:
for c in data.columns :
    if "Short" in c :
        del data[c]

In [ ]:
data

In [ ]:
print(data)

In [ ]:
#regarder quelles sont les colonnes qu'il faudra modifier car leurs valeurs sont exprimés avec des B (pour milliatrd) ou des M (pour million)
for c in data.columns :
    try :
        data[c] = data[c].map(lambda x : float(str(x).replace("k","")))
    except :
        print("colonne pas utilisable ", c)

In [ ]:
c = "Total Cash (mrq)"

In [ ]:
data[c] = np.where(data[c].str.contains("M"),data[c].map(lambda x : 1000000*float(str(x).replace("M","")) if ("M" in str(x)) else x),data[c])
data[c] = np.where(data[c].str.contains("B"),data[c].map(lambda x : 1000000000*float(str(x).replace("B","")) if ("B" in str(x)) else x),data[c])

In [ ]:
data.head(32)

In [ ]:
# Remplacer 2.80k par 2.80 dans la colonne Trailing PE ligne 31 (erreur de Yahoo Finance)
data['Trailing P/E '] = data['Trailing P/E '].replace("2.80k","2.80")

In [ ]:
PEdf=data[['Trailing P/E ', 'Symbol']]
print(PEdf)
PEdf = PEdf[~PEdf['Trailing P/E '].isna()]
print(PEdf)
print(PEdf.sort_values(by=[ 'Trailing P/E ', 'Symbol'], ascending=True))
#attention : un P/E négatif est mauvais signe donc LYFT ne peut pas être considéré comme prometteur avec cette analyse

In [ ]:
PEdf['Trailing P/E '].mean()
# The companies prefered with this criteria are those with a Trailing P/E value <=59

59.03535211267606

In [ ]:
#it’s a good sign that the forward P/E is lower than the trailing P/E, because that indicates this company’s earning is likely increase next year.
PEFdf=data[['Forward P/E ', 'Symbol']]
print(PEFdf)
PEFdf = PEFdf[~PEFdf['Forward P/E '].isna()]
print(PEFdf)

In [ ]:
#fusion de PEdf et PEFdf pour comparer si forward P/E < trailing P/E et rester avec l'ordre de PEdf
PEfusion=pd.merge(PEdf,PEFdf,how="left",on="Symbol")
PEfusion

In [ ]:
PEfusion =PEfusion.sort_values(by=['Trailing P/E '])


In [ ]:
PEfusion = PEfusion[~PEfusion['Forward P/E '].isna()]
print(PEfusion)

In [ ]:
np.where(PEfusion['Trailing P/E ']>PEfusion['Forward P/E '], True, False)

In [ ]:
PEfusion['conditionT/F']=np.where(PEfusion['Trailing P/E ']>PEfusion['Forward P/E '], True, False)

In [ ]:
PEfusion

In [ ]:
PEfusion['conditionmoyPET']=np.where(PEfusion['Trailing P/E ']<=59, True, False)

In [ ]:
PEfusion

In [ ]:
PEfusion[(PEfusion['conditionT/F'] == True) & (PEfusion['conditionmoyPET'] ==True)]

#on obtient notre df avec toutes les entreprises qui vérifient les 2 conditions avantageuses, les premières étant les meilleures selon condition 1 (cf analyse Trailing P/E)

In [ ]:
#PEG Ratio (5 yr expected) 
# When you compare two companies in the same industry using PEG PEG Ratio (5 yr expected) , simply put, the one with a lower value looks more attractive.
# Some investor believes that positive PEG Ratio (5 yr expected) lower than 1 are attractive (suggesting that stock is undervalued) and a PEG Ratio (5 yr expected)  of 2 or more indicates a stock is overvalued.
#colonne : 'PEG Ratio (5 yr expected) '

dfPEG=data[['PEG Ratio (5 yr expected) ', 'Symbol']]
print(dfPEG)
dfPEG = dfPEG[~dfPEG['PEG Ratio (5 yr expected) '].isna()]
print(dfPEG)
dfPEG=dfPEG.sort_values(by=[ 'PEG Ratio (5 yr expected) ', 'Symbol'], ascending=True)

In [ ]:
dfPEG['valeur<1']= np.where(dfPEG['PEG Ratio (5 yr expected) ']<1, True, False)

In [ ]:
dfPEG

In [ ]:
#P/S
#Many investors trust the P/S (Price/Sales) ratio more than the P/E (Price/Earning) ratio because unlike the earning number, the sales (revenue) number is less likely to be manipulated using accounting estimates and management manipulations.
#Generally speaking, when comparing two companies in the same industry, the one with a lower P/S ratio looks more attractive.
